In [ ]:
#Import packages 
import xarray as xr
import pyrams.xarray
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from glob import glob
from tqdm import tqdm
import pint_xarray

ureg = pint_xarray.unit_registry


In [ ]:
#open the simulation datasets
simlist = [
    'salt20',
    'salt200',
    'salt400',
    'salt600',
    'salt800',
    'salt1000',
]

sims = {}
for sim in simlist:
    data = xr.open_dataset(f'model_data/{sim}.nc')
    data = data.rams.apply_variable_metadata()
    data.time.attrs['units'] = 'hour'    
    sims[sim] = data
    
for name, ds in tqdm(sims.items()):
    ds = ds.pint.dequantify()

cmap = plt.get_cmap('jet', len(sims))

#open the sensitivity simulation datasets
simlist2 = [
    'salt400',
    'salt400-subs',
    'salt400-inv',
    'salt400-cool',
    'salt400-stable',
    'salt1000',
    'salt1000-subs',
    'salt1000-inv',
    'salt1000-cool',
    'salt1000-stable',
]

sims2 = {}
for sim in simlist2:
    data = xr.open_dataset(f'model_data/{sim}.nc')
    data = data.rams.apply_variable_metadata()
    data.time.attrs['units'] = 'hour'    
    sims2[sim] = data
    
for name, ds in tqdm(sims2.items()):
    ds = ds.pint.dequantify()

cmap2 = plt.get_cmap('tab10', 10)

plots_dir = './plots/'

abc='abcdefghij'

In [ ]:
#Initial* conditions - Need to make this pretty
#Actually, conditions at 0.5 hrs after some model spin up
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(10,3), constrained_layout=True, sharey=True)
#first two loop over sensitivity tests
for i,(name, ds) in enumerate(sims2.items()):
    ds = ds.isel(time=2)
    ds.THETA.plot(y='z', ax=ax1, label=name)
    if i==4: #because I don't know how to loop over just some of the simulations
        break
ax1.legend(loc='upper left', ncols=2, fontsize=8)
ax1.set_title(f'Time = {ds.time.values}h')
    
for i,(name, ds) in enumerate(sims2.items()):
    ds = ds.isel(time=2)
    ds.RV.pint.to('g/kg').plot(y='z', ax=ax2, label=name)
    if i==4:
        break
ax2.set_title(f'Time = {ds.time.values}h')
ax2.legend(loc='upper right', ncols=2, fontsize=8)

#loops over salt profiles
for i,(name, ds) in enumerate(sims.items()):
    ds = ds.isel(time=2)
    ds.SALT_FILM_NP.pint.to('1/mg').plot(y='z', ax=ax3,color=cmap(i), label=name)
    ax3.legend(ncols=2, fontsize=8)
ax3.set_title(f'Time = {ds.time.values}h')

ax1.set_xlabel('$\\theta$ (potential temperature) [K]')
ax2.set_xlabel('Water Vapor Mixing Ratio [g/kg]')
ax3.set_xlabel('Salt Number Concentration [mg$^{-1}$]]')
#and finally, save it
plt.savefig(f'{plots_dir}/02.png', dpi=500)

In [ ]:
fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(5,5))

for i,((name, ds), ax) in enumerate(zip(sims.items(), axs.flatten())):
    ds = ds.sel(time=slice(2,30))
    p = ds.RCP.where(ds.RCP >= 0.01 * ureg('g/kg')).pint.to('g/kg').plot(x='time', vmin=0, vmax=.35, ax=ax, add_colorbar=False) 
    ax.set_title(abc[i]+ ') ' + name)
    p.set_rasterized(True)
    if i%2 != 0: ax.set_ylabel('')
    else: ax.set_ylabel('Height [m]')
    if i not in [4,5]: ax.set_xlabel('')
    else: ax.set_xlabel('Time [hr]')
       
plt.ylim(0,1500)
plt.colorbar(p, ax=axs.ravel().tolist(), shrink=1, label='g kg$^{-1}$')
plt.savefig(f'{plots_dir}/03.png', dpi=500)

In [ ]:
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(10,3), constrained_layout=True)
for i,(name, ds) in enumerate(sims.items()):
    ds = ds.sel(time=slice(2,30))
    (ds.rams.lwp * ureg('m')).pint.to('g/m^2').plot(ax=ax1,x='time', label=name, color=cmap(i))

ax1.set_ylabel('LWP [g m$^{-2}$]')
ax1.set_xlabel('Time [hr]')
ax1.text(0.9, 0.9, 'a)', transform=ax1.transAxes)
    
for i,name in enumerate(sims.keys()):
    ds = sims[name]
    ds = ds.sel(time=slice(2,30))
    ((ds.RRP * ds.DN0).integrate('z') * ureg('m')).pint.to('g/m^2').plot(ax=ax2, label=name, color=cmap(i))

ax2.text(0.9,0.9,'b)',transform=ax2.transAxes)
ax2.set_xlabel('Time [hr]')
ax2.set_ylabel('RWP [g m$^{-2}$]')

for i,name in enumerate(sims.keys()):
    ds = sims[name].sel(time=slice(2,30))
    bl_mass = ds.SALT_FILM_NP.isel(z=0)*ds.DN0.isel(z=0)
    bl_mass.pint.to('1/cm^3').plot(ax=ax3, color=cmap(i), label=name)
    (ds.CCP*ds.DN0).where(ds.RCP >= 0.01 *ureg('g/kg')).mean('z').pint.to('1/cm^3').plot(ax=ax3,color=cmap(i), linestyle='--')
 
ax3.text(0.9,0.9,'c)',transform=ax3.transAxes)
ax3.set_ylabel('Number Concentration [# cm$^{-3}$]')
ax3.set_xlabel('Time [hr]')

plt.legend(ncol=1,bbox_to_anchor=(1.05, .8))

plt.savefig(f'{plots_dir}/04.png', dpi=500)

In [ ]:
fig, axs = plt.subplots(2, 4, sharey=True, constrained_layout=True, figsize=(8,4))
axs = axs.flatten()

times = [4,12,20,28]

for j in range(4):
#for t,ax in zip(times, axs):
    handles = []
    for i,(name, ds) in enumerate(sims.items()):
        ds1 = ds.sel(time=times[j])
        ax=axs[j]
        p = (ds1.CCP * ds1.DN0).pint.to('1/cm^3').plot(y='Z_NORM_CLOUD', ax=axs[j], color=cmap(i), label=name)
        ax.set_ylim(-0.1,1.1)
        ax.set_xlim(0,100)
        handles = handles + p
        ax.set_xlabel('Droplet Conc. [cm$^{-3}$]')
        ax.set_title(f'Time = {times[j]}.0 hr')
    if times[j] in [4]: 
        ax.set_ylabel('Norm. Cloud Hgt.')
    else: ax.set_ylabel('')
    
fig.supylabel('(a)\n\n\n\n\n\n\n\n\n\n\n(b)\n',rotation='horizontal')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#for t,ax in zip(times, axs):
for j in range(4):
    for i,(name, ds) in enumerate(sims.items()):
        ds1 = ds.sel(time=times[j])
        ax=axs[j+4]
        v = ds1.rams.cloudradius
        l = v.where(ds1.RCP >= 0.01 * ureg('g/kg')).pint.to('micron').plot(y='Z_NORM_CLOUD', ax=axs[j+4], color=cmap(i), label=name)
        ax.set_ylim(-0.1,1.1)
        # handles.append(p[0].get_label())
        handles = handles + l
        ax.set_xlabel('Droplet Radius [$\mu$m]')
        ax.set_xlim(0,25)
        ax.set_title('')
    if times[j] in [4]: ax.set_ylabel('Norm. Cloud Hgt.')
    else: ax.set_ylabel('')

plt.savefig(f'{plots_dir}/06.png', dpi=500)


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(6,6), constrained_layout=True)
axs=axs.flatten()

cp = 1004 * ureg('J/kg/K')

for i, (name,ds) in enumerate(sims.items()):
    ds = ds.sel(time=slice(2,30))
    v = -1* (ds.FTHRD * cp * ds.DN0).where(ds.Z_NORM_CLOUD <= 1,0).where(ds.Z_NORM_CLOUD >= 0,0).integrate('z') * ureg('m')
    v[v==0]=np.nan*ureg('J/s/m^2')
    v.pint.to('W/m^2').plot(ax=axs[0], label=name, color=cmap(i))
axs[0].set_ylabel('Cloud Rad. Flux Div. [W m$^{-2}$]')
axs[0].set_xlabel('Time [hr]')
axs[0].set_title('a) Cloud Rad. Flux Divergence')

for i,name in enumerate(sims.keys()):
    ds = sims[name]
    ds = ds.sel(time=30, z=slice(0,800))
    #uncomment later when VFLXW has proper attributes
    ds.VFLXW.plot(y='z', label=name, color=cmap(i),ax=axs[1])
    axs[1].set_xscale('log')
    # (ds.VFLXW).sel(time=slice(2,30)).mean('time').plot(label=name, y='z', color=cmap(i),ax=axs[1])
axs[1].set_xlabel('Vertical Wind Variance [m$^2$ s$^{-2}$]')
axs[1].set_title('b) Mean Wind Var.')
# axs[1].set_ylabel('Height [m]')
# axs[1].set_xscale('log')

for i,(name, ds) in enumerate(sims.items()):
    ds = ds.sel(time=slice(2,30))
    ds.BLTOP.plot(x='time', label=name, color=cmap(i),ax=axs[2])

axs[2].set_ylabel('Boundary Layer Top Height [m]')
axs[2].set_title('c) Boundary Layer Top')
axs[2].set_xlabel('Time [hr]')

for i,(name, ds) in enumerate(sims.items()):
    ds = ds.sel(time=30, z=slice(0,1200))
    ds.THETA.plot(y='z', label=name, color=cmap(i),ax=axs[3])
    
axs[3].set_xlabel('Potential Temperature [K]')
axs[3].set_ylabel('Height [m]')
axs[3].set_title('d) Potential Temperature at 28 hrs')
axs[3].legend()

plt.savefig(f'{plots_dir}/07.png', dpi=500)

In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, constrained_layout=True, figsize=(7.5,2.5))
axs = axs.flatten()

times = [4, 12, 20, 28]

for t,ax in zip(times, axs):
    handles = []
    for i,(name, ds) in enumerate(sims.items()):
        ds1 = ds.sel(time=t)
        l = (ds1.FTHRD).pint.to('K/hr').plot(y='Z_NORM_CLOUD', ax=ax, color=cmap(i), label=name)
        ax.set_ylim(.8,1.1)
        # handles.append(p[0].get_label())
        handles = handles + l
        ax.set_xlabel('Heat. Rate [K hr$^{-1}$]')
        ax.set_title(f'Time = {t}.0 hr')
    if t in [4]: ax.set_ylabel('Norm. Cloud Hgt.')
    else: ax.set_ylabel('')

fax = axs[-1]
labs = [l.get_label() for l in handles]
fax.legend(handles, labs, loc='center')
fax.set_axis_off()
plt.suptitle('Radiative Heating Rate')
plt.savefig(f'{plots_dir}/08.png', dpi=500)


In [ ]:
fig, axs = plt.subplots(2,3, sharey=True, constrained_layout=True, figsize=(7,5))

times = [4, 8, 12, 16, 20, 24, 28]
colors = plt.get_cmap('viridis', len(times))

for ((name,ds), ax) in zip(sims.items(), axs.flatten()):
    for i,t in enumerate(times):
        ds1 = ds.sel(time=t)
        #aero = (ds1.THETA)
        #aero.plot(y='Z_NORM_BLTOP', ax=ax, color=colors(i), label=t)
        aero = (ds1.SALT_FILM_NP)
        ax.set_ylim(0.9, 1.4)
        aero.pint.to('1/mg').plot(y='Z_NORM_BLTOP', ax=ax, color=colors(i), label=t)
    
    ax.axhline(1, color='k', linestyle='--')
    ax.set_title(name)
    ax.set_ylabel('')   
    ax.set_xlabel('')

axs[1,0].set_xlabel('Salt Conc. [# mg$^{-3}$]')    
axs[1,1].set_xlabel('Salt Conc. [# mg$^{-3}$]') 
axs[1,2].set_xlabel('Salt Conc. [# mg$^{-3}$]') 
axs[0,0].set_ylabel('Norm. BL Height')
axs[1,0].set_ylabel('Norm. BL Height')
axs[0,0].legend(ncol=3, title='Hour', fontsize=8)
plt.savefig(f'{plots_dir}/09.png', dpi=500)

In [ ]:
cmap2

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(10,3), constrained_layout=True)
for i,(name, ds) in enumerate(sims2.items()):
    ds = ds.sel(time=slice(1,30))
    if i<5:
        (ds.rams.lwp * ureg('m')).pint.to('g/m^2').plot(ax=ax1,x='time', label=name, color=cmap2(i))
    else:
        (ds.rams.lwp * ureg('m')).pint.to('g/m^2').plot(ax=ax1,x='time', label=name, color=cmap2(i-5), linestyle='--')
        
ax1.set_ylabel('LWP [g m$^{-2}$]')
ax1.set_xlabel('Time [hr]')
ax1.text(0.9, 0.9, 'a)', transform=ax1.transAxes)
    
for i,name in enumerate(sims2.keys()):
    ds = sims2[name]
    ds = ds.sel(time=slice(1,30))
    if i<5:
        ((ds.RRP * ds.DN0).integrate('z') * ureg('m')).pint.to('g/m^2').plot(ax=ax2, label=name, color=cmap2(i))
    else: 
        ((ds.RRP * ds.DN0).integrate('z') * ureg('m')).pint.to('g/m^2').plot(ax=ax2, label=name, color=cmap2(i-5),linestyle='--')
ax2.text(0.9,0.9,'b)',transform=ax2.transAxes)
ax2.set_xlabel('Time [hr]')
ax2.set_ylabel('RWP [g m$^{-2}$]')

for i,name in enumerate(sims2.keys()):
    ds = sims2[name].sel(time=slice(1,30))
    if i<5:
        (ds.CCP*ds.DN0).where(ds.RCP >= 0.01 *ureg('g/kg')).mean('z').pint.to('1/cm^3').plot(ax=ax3,color=cmap2(i), label=name)
    else:
        (ds.CCP*ds.DN0).where(ds.RCP >= 0.01 *ureg('g/kg')).mean('z').pint.to('1/cm^3').plot(ax=ax3,color=cmap2(i-5), linestyle='--', label=name)
ax3.text(0.9,0.9,'c)',transform=ax3.transAxes)
ax3.set_xlabel('Time [hr]')
ax3.set_ylabel('Droplet Conc. [# cm$^{-3}$]')
plt.legend(ncol=1,bbox_to_anchor=(1.05, 1))

# plt.savefig(f'{plots_dir}/04.png', dpi=500)